In [1]:
HOME_PATH = "/home/tiny-faces"

ROOT="data/WIDER"
TRAINDATA=ROOT + "/wider_face_split/wider_face_train_bbx_gt.txt"
VALDATA=ROOT + "/wider_face_split/wider_face_val_bbx_gt.txt"
TESTDATA=ROOT + "/wider_face_split/wider_face_test_filelist.txt"
CHECKPOINT = "weights/checkpoint_50_best.pth"

%cd "$HOME_PATH"

/home/tiny-faces


In [2]:
!pip list | grep pyclust

pyclust                       0.2.0
pyclustering                  0.10.1.2


In [3]:
!which pip

/opt/conda/bin/pip


In [4]:
!which python

/opt/conda/bin/python


In [5]:
!python evaluate.py "$TESTDATA" --dataset-root "$ROOT" --checkpoint "$CHECKPOINT" --split test

Dataset loaded
16097 samples in the test dataset
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|████████████████████████████████████████| 171M/171M [00:15<00:00, 11.7MB/s]
 34%|████████████▎                       | 5507/16097 [39:18<1:15:34,  2.34it/s]
Traceback (most recent call last):
  File